In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from googleapiclient.discovery import build


import httpx

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

load_dotenv()
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())
model_name = "gpt-3.5-turbo-0125"

google_cse_id = os.environ['GOOGLE_CSE_ID']
google_api_key = os.environ['GOOGLE_API_KEY']